In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('./data/preprocessed_Bank_Customer_Churn_Prediction.csv')

In [3]:
df


,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,0,1,42,2,0.00,1,1,1,101348.88,1
1,608,2,1,41,1,83807.86,1,0,1,112542.58,0
2,502,0,1,42,8,159660.80,3,1,0,113931.57,1
3,699,0,1,39,1,0.00,2,0,0,93826.63,0
4,850,2,1,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,0,39,5,0.00,2,1,0,96270.64,0
9996,516,0,0,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,1,36,7,0.00,1,0,1,42085.58,1
9998,772,1,0,42,3,75075.31,2,1,0,92888.52,1


### XGBOOST 사용

In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42, stratify=y)

# XGBoost 모델 생성 및 학습
model = xgb.XGBClassifier(
    objective='binary:logistic', 
    eval_metric='logloss', 
    use_label_encoder=False, 
    random_state=42
)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


Accuracy: 0.8520
Classification Report:
               precision    recall  f1-score   support

           0     0.8813    0.9408    0.9101      2787
           1     0.6857    0.5049    0.5816       713

    accuracy                         0.8520      3500
   macro avg     0.7835    0.7229    0.7458      3500
weighted avg     0.8415    0.8520    0.8432      3500



c:\Users\Playdata\AppData\Local\anaconda3\envs\psystudy_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:35:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [5]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report

# 데이터 준비
y = df['churn']
X = df.drop(columns=['churn'])

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42, stratify=y)

# 하이퍼파라미터 설정
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'random_state': 42,
    'learning_rate': 0.1,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'n_estimators': 100
}

# ✅ XGBoost의 cv 기능 사용
cv_results = xgb.cv(
    params=params,
    dtrain=xgb.DMatrix(X_train, label=y_train),
    num_boost_round=100,
    nfold=5,
    stratified=True,
    metrics='logloss',
    early_stopping_rounds=10
)

# 최적 n_estimators 설정
best_n_estimators = len(cv_results)
params['n_estimators'] = best_n_estimators

# 최적 모델 학습
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)

print(f'Best n_estimators: {best_n_estimators}')
print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


c:\Users\Playdata\AppData\Local\anaconda3\envs\psystudy_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:35:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Best n_estimators: 73
Accuracy: 0.8643
Classification Report:
               precision    recall  f1-score   support

           0     0.8790    0.9620    0.9186      2787
           1     0.7644    0.4825    0.5916       713

    accuracy                         0.8643      3500
   macro avg     0.8217    0.7222    0.7551      3500
weighted avg     0.8557    0.8643    0.8520      3500



### LGB사용

In [6]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42, stratify=y)

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)  # 검증 데이터 설정

# LightGBM 모델 학습
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbosity': -1
}

# callbacks을 사용하여 early stopping 적용
model = lgb.train(params, train_data, num_boost_round=100,
                  valid_sets=[valid_data], valid_names=['valid'],
                  callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])

# 예측
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)


print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


Training until validation scores don't improve for 10 rounds
[10]	valid's binary_logloss: 0.369569
[20]	valid's binary_logloss: 0.34421
[30]	valid's binary_logloss: 0.336481
[40]	valid's binary_logloss: 0.333423
[50]	valid's binary_logloss: 0.3331
[60]	valid's binary_logloss: 0.333744
Early stopping, best iteration is:
[55]	valid's binary_logloss: 0.332668
Accuracy: 0.8634
Classification Report:
               precision    recall  f1-score   support

           0     0.8824    0.9559    0.9177      2787
           1     0.7443    0.5021    0.5997       713

    accuracy                         0.8634      3500
   macro avg     0.8133    0.7290    0.7587      3500
weighted avg     0.8543    0.8634    0.8529      3500



### LGB gridsearch사용

In [7]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42, stratify=y)

# LightGBM 모델 객체 생성
model = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', verbosity=-1)

# 하이퍼파라미터 검색 공간 정의
param_grid = {
    'num_leaves': [20, 31, 50],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_samples': [10, 20, 50],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Grid Search 수행
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,  # 3-폴드 교차검증
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# 최적 하이퍼파라미터 출력
print("Best parameters found:", grid_search.best_params_)

# 최적 하이퍼파라미터 적용하여 모델 재학습
best_model = grid_search.best_estimator_

# 예측
y_pred = best_model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred,digits=4)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


Fitting 3 folds for each of 729 candidates, totalling 2187 fits
Best parameters found: {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_samples': 20, 'num_leaves': 20, 'subsample': 0.7}
Accuracy: 0.8651
Classification Report:
               precision    recall  f1-score   support

           0     0.8779    0.9648    0.9193      2787
           1     0.7757    0.4755    0.5896       713

    accuracy                         0.8651      3500
   macro avg     0.8268    0.7201    0.7544      3500
weighted avg     0.8571    0.8651    0.8521      3500



---

# optuna 사용


In [8]:
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report



# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42, stratify=y)

# Optuna를 이용한 하이퍼파라미터 튜닝
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    model = lgb.train(params, train_data, num_boost_round=100,
                      valid_sets=[valid_data], 
                      callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])
    
    y_pred = model.predict(X_test)
    y_pred = [1 if x > 0.5 else 0 for x in y_pred]
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# 최적 하이퍼파라미터 출력
print("Best parameters found:", study.best_params)

# 최적 하이퍼파라미터로 모델 학습
best_params = study.best_params
best_params.update({'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1})

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

model = lgb.train(best_params, train_data, num_boost_round=100,
                  valid_sets=[valid_data], 
                  callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])

# 최적 모델 예측
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


[I 2025-02-08 20:38:53,170] A new study created in memory with name: no-name-4d58d944-59d1-46f4-aaf5-25fbd3d30ea4


Training until validation scores don't improve for 10 rounds

C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:27: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)



[10]	valid_0's binary_logloss: 0.405879
[20]	valid_0's binary_logloss: 0.36831
[30]	valid_0's binary_logloss: 0.351333
[40]	valid_0's binary_logloss: 0.341761
[50]	valid_0's binary_logloss: 0.337051
[60]	valid_0's binary_logloss: 0.334523
[70]	valid_0's binary_logloss: 0.332716
[80]	valid_0's binary_logloss: 0.332041
[90]	valid_0's binary_logloss: 0.332037
Early stopping, best iteration is:
[89]	valid_0's binary_logloss: 0.331811
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.355473
[20]	valid_0's binary_logloss: 0.337543
[30]	valid_0's binary_logloss: 0.335504
[40]	valid_0's binary_logloss: 0.331929
[50]	valid_0's binary_logloss: 0.332144
Early stopping, best iteration is:
[43]	valid_0's binary_logloss: 0.331433
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.345473
[20]	valid_0's binary_logloss: 0.333187
[30]	valid_0's binary_logloss: 0.335882
Early stopping, best iteration is:
[20]	valid_0'

C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:27: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)


...

Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.477887
[20]	valid_0's binary_logloss: 0.455352
[30]	valid_0's binary_logloss: 0.439191
[40]	valid_0's binary_logloss: 0.423244
[50]	valid_0's binary_logloss: 0.412745
[60]	valid_0's binary_logloss: 0.401547
[70]	valid_0's binary_logloss: 0.394709
[80]	valid_0's binary_logloss: 0.385743
[90]	valid_0's binary_logloss: 0.379271


[I 2025-02-08 20:38:58,284] Trial 27 finished with value: 0.846 and parameters: {'num_leaves': 102, 'max_depth': 14, 'learning_rate': 0.015540582678389623, 'min_child_samples': 57, 'subsample': 0.5842287112018226, 'colsample_bytree': 0.5423908938251035}. Best is trial 26 with value: 0.8677142857142857.
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Lo

[100]	valid_0's binary_logloss: 0.374102
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.374102
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.459251
[20]	valid_0's binary_logloss: 0.428903
[30]	valid_0's binary_logloss: 0.408198
[40]	valid_0's binary_logloss: 0.389153
[50]	valid_0's binary_logloss: 0.377679
[60]	valid_0's binary_logloss: 0.368255
[70]	valid_0's binary_logloss: 0.361221
[80]	valid_0's binary_logloss: 0.354228
[90]	valid_0's binary_logloss: 0.349357
[100]	valid_0's binary_logloss: 0.346292
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.346292


C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_18140\3189017698.py:27: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)


Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.406236
[20]	valid_0's binary_logloss: 0.365687
[30]	valid_0's binary_logloss: 0.351779
[40]	valid_0's binary_logloss: 0.343164
[50]	valid_0's binary_logloss: 0.33982
[60]	valid_0's binary_logloss: 0.337939
[70]	valid_0's binary_logloss: 0.337754
Early stopping, best iteration is:
[68]	valid_0's binary_logloss: 0.337173
Best parameters found: {'num_leaves': 44, 'max_depth': 7, 'learning_rate': 0.02621764072268792, 'min_child_samples': 81, 'subsample': 0.5586478288101614, 'colsample_bytree': 0.621088134499706}
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.459251
[20]	valid_0's binary_logloss: 0.428903
[30]	valid_0's binary_logloss: 0.408198
[40]	valid_0's binary_logloss: 0.389153
[50]	valid_0's binary_logloss: 0.377679
[60]	valid_0's binary_logloss: 0.368255
[70]	valid_0's binary_logloss: 0.361221
[80]	valid_0's binary_logloss: 0.354228
[90]	valid_

In [9]:
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42, stratify=y)

# Optuna를 이용한 하이퍼파라미터 튜닝
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    model = lgb.train(params, train_data, num_boost_round=100,
                      valid_sets=[valid_data], 
                      callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])
    
    y_pred_proba = model.predict(X_test)
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
    best_threshold = thresholds[(precision + recall).argmax()]  # 최적의 threshold 선택
    
    y_pred = [1 if x > best_threshold else 0 for x in y_pred_proba]
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# 최적 하이퍼파라미터 출력
print("Best parameters found:", study.best_params)

# 최적 하이퍼파라미터로 모델 학습
best_params = study.best_params
best_params.update({'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1})

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

model = lgb.train(best_params, train_data, num_boost_round=100,
                  valid_sets=[valid_data], 
                  callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])

# 최적 모델 예측
y_pred_proba = model.predict(X_test)
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
best_threshold = thresholds[(precision + recall).argmax()]  # 최적 threshold 선택

print(f'Best Threshold: {best_threshold:.4f}')

y_pred = [1 if x > best_threshold else 0 for x in y_pred_proba]

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


[I 2025-02-08 20:38:58,818] A new study created in memory with name: no-name-76c03788-e975-4a8d-be97-d16bc9e0489b


Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.371672
[20]	valid_0's binary_logloss: 0.34107
[30]	valid_0's binary_logloss: 0.335414


[I 2025-02-08 20:38:58,937] Trial 0 finished with value: 0.79 and parameters: {'num_leaves': 52, 'max_depth': 14, 'learning_rate': 0.13156384122674317, 'min_child_samples': 78, 'subsample': 0.8742318605350777, 'colsample_bytree': 0.6702172595944933}. Best is trial 0 with value: 0.79.


[40]	valid_0's binary_logloss: 0.337333
Early stopping, best iteration is:
[34]	valid_0's binary_logloss: 0.33507
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.476114
[20]	valid_0's binary_logloss: 0.452793
[30]	valid_0's binary_logloss: 0.43474
[40]	valid_0's binary_logloss: 0.419687
[50]	valid_0's binary_logloss: 0.407498
[60]	valid_0's binary_logloss: 0.39763
[70]	valid_0's binary_logloss: 0.38922


[I 2025-02-08 20:38:59,219] Trial 1 finished with value: 0.758 and parameters: {'num_leaves': 149, 'max_depth': 10, 'learning_rate': 0.010970977337149948, 'min_child_samples': 88, 'subsample': 0.5216977061809234, 'colsample_bytree': 0.9531785211453723}. Best is trial 0 with value: 0.79.


...

[30]	valid_0's binary_logloss: 0.377187
[40]	valid_0's binary_logloss: 0.361843
[50]	valid_0's binary_logloss: 0.35219
[60]	valid_0's binary_logloss: 0.345092
[70]	valid_0's binary_logloss: 0.340381
[80]	valid_0's binary_logloss: 0.335867
[90]	valid_0's binary_logloss: 0.334054
[100]	valid_0's binary_logloss: 0.333232
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.333232
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.466615
[20]	valid_0's binary_logloss: 0.438285
[30]	valid_0's binary_logloss: 0.419496
[40]	valid_0's binary_logloss: 0.402527
[50]	valid_0's binary_logloss: 0.391082


[I 2025-02-08 20:39:03,820] Trial 28 finished with value: 0.868 and parameters: {'num_leaves': 87, 'max_depth': 5, 'learning_rate': 0.023834120605813875, 'min_child_samples': 73, 'subsample': 0.9233391711145824, 'colsample_bytree': 0.5034615458480549}. Best is trial 20 with value: 0.8691428571428571.


[60]	valid_0's binary_logloss: 0.380002
[70]	valid_0's binary_logloss: 0.37345
[80]	valid_0's binary_logloss: 0.365541
[90]	valid_0's binary_logloss: 0.359539
[100]	valid_0's binary_logloss: 0.355382
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.355382
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.473134
[20]	valid_0's binary_logloss: 0.448846
[30]	valid_0's binary_logloss: 0.427898
[40]	valid_0's binary_logloss: 0.412332
[50]	valid_0's binary_logloss: 0.400491
[60]	valid_0's binary_logloss: 0.389769
[70]	valid_0's binary_logloss: 0.38053


[I 2025-02-08 20:39:04,013] Trial 29 finished with value: 0.7951428571428572 and parameters: {'num_leaves': 116, 'max_depth': 7, 'learning_rate': 0.015596101682981074, 'min_child_samples': 78, 'subsample': 0.6922678696789265, 'colsample_bytree': 0.6667067229068658}. Best is trial 20 with value: 0.8691428571428571.


[80]	valid_0's binary_logloss: 0.371975
[90]	valid_0's binary_logloss: 0.365795
[100]	valid_0's binary_logloss: 0.360828
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.360828
Best parameters found: {'num_leaves': 88, 'max_depth': 8, 'learning_rate': 0.02015953109992848, 'min_child_samples': 75, 'subsample': 0.6842691627142414, 'colsample_bytree': 0.5383316557129085}
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.471173
[20]	valid_0's binary_logloss: 0.444564
[30]	valid_0's binary_logloss: 0.426505
[40]	valid_0's binary_logloss: 0.40969
[50]	valid_0's binary_logloss: 0.398563
[60]	valid_0's binary_logloss: 0.387513
[70]	valid_0's binary_logloss: 0.381028
[80]	valid_0's binary_logloss: 0.372571
[90]	valid_0's binary_logloss: 0.366219
[100]	valid_0's binary_logloss: 0.361697
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.361697
Best Threshold: 0.4494
Accuracy: 0.8691
Classi

Precision과 Recall 균형 조정

Precision 증가(0.80 → 0.84)
Recall 감소(0.48 → 0.44)
이탈 고객을 보다 정확하게 예측하지만, 일부를 놓칠 가능성이 있음
비즈니스 목표에 따라 Threshold를 조정할 필요 있음!
→ 이탈 고객 Recall을 높이고 싶다면 더 낮은 Threshold를 시도하는 것도 가능